# PTB on TensorFlow

Step 1: Import packages.

In [1]:
import numpy as np
import sys
import os
import tensorflow as tf

print(tf.__version__)

1.10.0


Step 2: Define constants.

In [2]:
tf.app.flags.DEFINE_string('f', '', 'kernel')

data_dir = '../data/ptb'

tf.flags.DEFINE_integer("train_batch_size", 20, "batch size of training")
tf.flags.DEFINE_integer("train_num_steps", 35, "num of layers of training")
tf.flags.DEFINE_string("train_data", os.path.join(data_dir, "ptb.train"), "filename of training data")
tf.flags.DEFINE_integer("num_layers", 2, "num of layers")
tf.flags.DEFINE_integer("lstm_size", 30, "hidden size of lstm")
tf.flags.DEFINE_float("lstm_keep_prob", 0.9, "keep prob of training")
tf.flags.DEFINE_float("embedding_keep_prob", 0.9, "keep prob of embedding")
tf.flags.DEFINE_integer("vocab_size", 10000, "the number of different words")
tf.flags.DEFINE_boolean("share_emb_and_softmax", False, "share the weights of embedding layer and softmax layer")
tf.flags.DEFINE_integer("max_grad_norm", 5, "control grad's value")
tf.flags.DEFINE_float("learning_rate", 1.0, "learning rate")
tf.flags.DEFINE_integer("eval_batch_size", 1, "batch size of training")
tf.flags.DEFINE_integer("eval_num_steps", 1, "num of layers of training")
tf.flags.DEFINE_string("eval_data", os.path.join(data_dir, "ptb.eval"), "filename of eval data")
tf.flags.DEFINE_string("test_data", os.path.join(data_dir, "ptb.test"), "filename of test data")
tf.flags.DEFINE_integer("num_epochs", 5, "num of epochs")


FLAGS = tf.flags.FLAGS
FLAGS(sys.argv) # 启用flags

['/home/orris/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py']

Step 3: Prepare data.

In [3]:
def make_batches(data_file, batch_size, num_steps):
    # put together a large sentence
    with open(data_file,'r') as f:
        sentence = ' '.join([line.strip() for line in f.readlines()])
    # split into [int, ]
    data = np.array(sentence.strip().split())
    # calculate number of batches (len(data) - 1, because we need an extra elm)
    num_batches = (len(data) - 1 ) // (batch_size * num_steps)

    ###############################################################################
    # features
    ###############################################################################
    # capture the first (num_batches * batch_size * num_steps) elms
    features = data[:(num_batches * batch_size * num_steps)]
    # split into num_batches batches => (num_batches, batch_size * num_steps)
    features = np.split(features, num_batches)
    # reshape => [num_batches, batch_size, num_steps]
    features = np.reshape(features, [num_batches, batch_size, num_steps])
     
    ###############################################################################
    # label
    ###############################################################################
    # capture the first elm to (num_batches * batch_size * num_steps + 1) elms
    labels = data[1:(num_batches * batch_size * num_steps + 1)]
    # split into num_batches batches => (num_batches, batch_size * num_steps)
    labels = np.split(labels, num_batches)
    # reshape => [num_batches, batch_size, num_steps]
    labels = np.reshape(labels, [num_batches, batch_size, num_steps])

    # return zip (features,label)
    return list(zip(features, labels)) # [num_batches, batch_size, num_steps]


# get train_batches
train_batches = make_batches(FLAGS.train_data,FLAGS.train_batch_size,FLAGS.train_num_steps)
# get eval_batches
eval_batches = make_batches(FLAGS.eval_data,FLAGS.eval_batch_size,FLAGS.eval_num_steps)
# get test_batches
test_batches = make_batches(FLAGS.test_data,FLAGS.eval_batch_size,FLAGS.eval_num_steps)


Step 4: Build models.

In [4]:
'''
    Train a model that have high fitting degree with PTB dataset
'''

class PTBModel(object):
    def __init__(self,num_steps, lstm_size, vocab_size, num_layers, share_emb_and_softmax, is_training, learning_rate, max_grad_norm,batch_size):
        with tf.name_scope('placeholder'):
            # features placeholder:[batch_size, num_steps]
            self.features = tf.placeholder(tf.int32,[None,num_steps], name='features') 
            # labels placeholder:[batch_size, num_steps]
            self.labels = tf.placeholder(tf.int32,[None,num_steps], name='labels')

            self.keep_prob = tf.placeholder(tf.float32, name='keep_prob')

        self.num_steps = num_steps
        self.lstm_size = lstm_size
        self.vocab_size = vocab_size
        self.num_layers = num_layers
        self.share_emb_and_softmax = share_emb_and_softmax
        self.is_training = is_training
        self.learning_rate = learning_rate
        self.max_grad_norm = max_grad_norm
        
        
        ###############################################################################
        # embedding
        ###############################################################################
        # assign lstm_size to embedding_size
        embedding_size = lstm_size

        # define an embedding matrix: [vocab_size, embedding_size(lstm_size)]
        W_embedding = tf.get_variable(name="W_embedding", shape=[vocab_size,embedding_size])

        # embedding: => [batch_size, num_steps, lstm_size]
        embedded_chars = tf.nn.embedding_lookup(W_embedding, self.features)

        # dropout
        # new
        if is_training:
            embedded_chars = tf.nn.dropout(embedded_chars,FLAGS.embedding_keep_prob)
        #
        #
        ###############################################################################
        # lstm
        ###############################################################################
        # define a multiple LSTM cell
        def create_lstm_cell(lstm_size, output_keep_prob):
            lstm_cell=tf.contrib.rnn.BasicLSTMCell(lstm_size,state_is_tuple=True)
            lstm_cell = tf.nn.rnn_cell.DropoutWrapper(lstm_cell, output_keep_prob=output_keep_prob)
            return lstm_cell

        lstm_cell = tf.nn.rnn_cell.MultiRNNCell([create_lstm_cell(lstm_size, self.keep_prob) for _ in range(num_layers)])

        # dynamic_run: outputs=[batch_size, num_steps, lstm_size]
        outputs,final_state = tf.nn.dynamic_rnn(lstm_cell,embedded_chars,dtype=tf.float32)


        ###############################################################################
        # softmax => [batch_size, num_steps, vocab_size]
        ###############################################################################
        # if share weights with embedding layer:
        if share_emb_and_softmax:
            # #W_softmax <= get_variable('embedding'): [vocab_size, embedding_size]
            # directly transpose W_embedding
            W_softmax = tf.transpose(W_embedding)
            # transpose W_softmax: => [lstm_size, vocab_size]
        else:
            W_softmax = tf.get_variable(name='W_softmax',shape=[lstm_size, vocab_size])
        # define a bias: [vocab_size]
        b_softmax = tf.get_variable(name="b_softmax", shape=[vocab_size])


        # get outputs: reshape output into [batch_size * num_steps, lstm_size]
        outputs = tf.reshape(outputs, [-1, lstm_size])
        #outputs = tf.reshape(tf.concat(outputs,1),[-1,lstm_size])


        # outputs * W + b: => [batch_size, num_steps, vocab_size]
        y_predicted = tf.matmul(outputs,W_softmax) + b_softmax
       
        # get labels: reshape self.labels into [batch_size * num_steps]
        # calc loss
        self.loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels = tf.reshape(self.labels, [-1]),logits = y_predicted))

        #self.loss = tf.reduce_sum(tf.nn.sparse_softmax_cross_entropy_with_logits(labels = tf.reshape(self.labels, [-1]),logits = y_predicted)) / batch_size
        

        # if not is_training:
        if not is_training:
            # return 
            return 

        # calc gradients
        params = tf.trainable_variables()
        opt = tf.train.GradientDescentOptimizer(learning_rate)
        gradients = tf.gradients(self.loss * num_steps, params)
        # clip by grad norm
        clipped_gradients, norm = tf.clip_by_global_norm(gradients,max_grad_norm)
        # apply gradients
        self.train = opt.apply_gradients(zip(clipped_gradients, params))

# define an initializer
initializer = tf.random_uniform_initializer(-0.05, 0.05)

# with tf.variable_scope('model',reuse=False,initializer)
with tf.variable_scope('model',reuse=False,initializer=initializer):
    # construct a train model object
    train_model = PTBModel(FLAGS.train_num_steps, FLAGS.lstm_size, FLAGS.vocab_size, FLAGS.num_layers, FLAGS.share_emb_and_softmax, True, FLAGS.learning_rate, FLAGS.max_grad_norm,FLAGS.train_batch_size)
# with tf.variable_scope('model',reuse=True,initializer)
with tf.variable_scope('model',reuse=True,initializer=initializer):
    # construct a eval model object: eval model also used as test model
    eval_model = PTBModel(FLAGS.eval_num_steps, FLAGS.lstm_size, FLAGS.vocab_size, FLAGS.num_layers, FLAGS.share_emb_and_softmax, False, FLAGS.learning_rate, FLAGS.max_grad_norm,FLAGS.eval_batch_size)

Step 5: Train model and evaluate it.

In [5]:
def run_epoch(sess, model, batches, keep_prob, train_op, is_training):
    '''
        train, evaluate and test the model
    '''
    # define total_loss: 0.0
    total_loss = 0.0
    # define step: 0
    step = 0
    iters = 0
    # for X, y in batches:
    for X, y in batches:
        # update step
        step += 1
        # run train_op and loss_op
        if is_training:
            _, loss= sess.run([model.train, model.loss],feed_dict={model.features:X, model.labels:y, model.keep_prob:keep_prob})
        else:
            loss = sess.run(model.loss,feed_dict={model.features:X, model.labels:y, model.keep_prob:keep_prob})
        iters += model.num_steps

        # add loss_op to total_loss
        total_loss += loss
        # divide total_loss by step to get loss_per_word
        loss_per_word = total_loss / step
        # calc perplexity: np.exp(loss_per_word)
        perplexity = np.exp(loss_per_word)
        # print perplexity
        if step % 100 == 0:
            #print('step {0}:{1}'.format(step,perplexity))
            print('step {0}:{1}'.format(step,np.exp(total_loss / step)))

    # return perplexity
    return perplexity


config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(FLAGS.num_epochs):
        # run train epoch
        train_perplexity = run_epoch(sess, train_model, train_batches, FLAGS.lstm_keep_prob, train_model.train, True)
        # print train perplexity
        print('epoch {0}: perplexity={1}'.format(epoch, train_perplexity))
        # run eval epoch
        eval_perplexity = run_epoch(sess, eval_model, eval_batches, 1, tf.no_op(), False)
        # print eval perplexity
        print('epoch {0}: perplexity={1}'.format(epoch, eval_perplexity))

step 100:1312.476784894744
step 200:1066.8174761198827
step 300:986.0881946164593
step 400:916.1572698616332
step 500:853.7472021448466
step 600:795.0659696491784
step 700:744.1981742146562
step 800:697.3388468160197
step 900:661.723710486976
step 1000:631.3018854967564
step 1100:603.0537741048239
step 1200:578.5981993751511
step 1300:554.9549505633757
epoch 0: perplexity=550.2413708862605
step 100:717.6536840407568
step 200:638.7110081278386
step 300:673.3060875296808
step 400:621.5795904161931
step 500:610.2699708850012
step 600:601.762724217218
step 700:595.8121998845381
step 800:599.6331069452566
step 900:609.7981458711796
step 1000:558.2837570734421
step 1100:587.773184947634
step 1200:604.5888193470828
step 1300:599.4843460747385
step 1400:580.3379993283037
step 1500:558.5687618410864
step 1600:546.5324018438179
step 1700:536.054491448928
step 1800:547.6557911724681
step 1900:553.7353923004886
step 2000:536.2688941865023
step 2100:544.28032176395
step 2200:544.4522276516979
step 

step 27100:398.9499273059852
step 27200:398.6953643088448
step 27300:398.1016358378906
step 27400:398.3897456013721
step 27500:397.9933947513867
step 27600:397.6803838759138
step 27700:397.451026367529
step 27800:397.6872732269379
step 27900:398.5715655954431
step 28000:399.23542264295986
step 28100:399.3334869527754
step 28200:399.4853882178508
step 28300:400.02840856152284
step 28400:399.81486894008185
step 28500:399.6931921419905
step 28600:399.9265785885801
step 28700:399.83831647605854
step 28800:399.66340749856647
step 28900:399.5043254587529
step 29000:399.0190656367616
step 29100:399.51137820988265
step 29200:399.73087398355466
step 29300:400.23472941765743
step 29400:400.6377426174442
step 29500:400.96371125715586
step 29600:400.962437540489
step 29700:400.76473345976626
step 29800:400.6312590831547
step 29900:401.34199402606635
step 30000:401.31724265692924
step 30100:400.9614310534395
step 30200:400.943019029034
step 30300:401.2654404278441
step 30400:401.3494742514903
step 

step 55100:401.14197462006274
step 55200:401.54823326622557
step 55300:401.47768908875463
step 55400:401.24184630724756
step 55500:400.59416508714014
step 55600:400.28330431826646
step 55700:400.36341812794336
step 55800:400.21846000674475
step 55900:400.22647033219005
step 56000:400.6303622316381
step 56100:400.9792769450822
step 56200:401.0620712402114
step 56300:401.25061763093964
step 56400:401.6219831929009
step 56500:401.6197373102243
step 56600:401.62112142445716
step 56700:401.7098531292262
step 56800:401.5466364640622
step 56900:401.2200425413352
step 57000:400.6735142190024
step 57100:400.49844730883984
step 57200:400.6089943112922
step 57300:400.44046570146617
step 57400:400.3640199508532
step 57500:400.3103175348062
step 57600:400.300461115385
step 57700:400.0906072407091
step 57800:400.2811002281366
step 57900:400.49981618695796
step 58000:400.71844559398744
step 58100:400.6801958402948
step 58200:400.2673276799665
step 58300:400.0645448134497
step 58400:399.41287391849255

step 8200:350.68496832161435
step 8300:349.8205952560946
step 8400:350.6011201410809
step 8500:354.2239142639476
step 8600:353.2674411352707
step 8700:353.017322005557
step 8800:354.34559430347593
step 8900:355.4092899891812
step 9000:355.3042657596419
step 9100:354.2974753541501
step 9200:357.21059759779587
step 9300:358.675448396556
step 9400:357.60846386696244
step 9500:358.9130139728687
step 9600:361.2697463521866
step 9700:361.91608349674823
step 9800:363.6316950982111
step 9900:363.3669239975817
step 10000:362.07080640775075
step 10100:361.5317222834055
step 10200:360.6284590040259
step 10300:359.72334443165425
step 10400:359.98361298337585
step 10500:359.0785306354899
step 10600:359.0699128136755
step 10700:359.95298472266825
step 10800:358.56772772007304
step 10900:359.8159143848259
step 11000:358.5660441926016
step 11100:358.95040144568526
step 11200:358.5001323787996
step 11300:357.5270679650698
step 11400:357.5033234975746
step 11500:357.86736841752776
step 11600:357.4126668

step 36400:326.53679431062017
step 36500:326.451452933385
step 36600:326.9621826348282
step 36700:326.84732978290606
step 36800:327.2812416415681
step 36900:327.9895446894558
step 37000:327.8483249894011
step 37100:327.3841368420965
step 37200:327.1184730640859
step 37300:327.03239337818275
step 37400:327.40499120741816
step 37500:327.9628063433622
step 37600:328.3501661130423
step 37700:328.3394302090291
step 37800:328.96419632210745
step 37900:329.0955674198009
step 38000:329.1477102532718
step 38100:329.0266586308625
step 38200:329.027700597365
step 38300:329.382702601722
step 38400:329.5083435710775
step 38500:329.74744250774603
step 38600:329.85977055291016
step 38700:329.8703903226072
step 38800:329.7948410598965
step 38900:329.99350350794595
step 39000:329.81332952284316
step 39100:329.8070977137325
step 39200:330.0306292438479
step 39300:329.9369656993474
step 39400:330.26605441201684
step 39500:330.32502095056134
step 39600:329.8247900336715
step 39700:329.297362696727
step 39

...

step 26800:308.3608778746168
step 26900:307.01701461586225
step 27000:305.7981133389611
step 27100:304.9941198943804
step 27200:304.9104151187074
step 27300:304.4159462263441
step 27400:304.5954742384096
step 27500:304.1496500296845
step 27600:303.8410999727115
step 27700:303.49816725501285
step 27800:303.7518734732811
step 27900:304.3671026726532
step 28000:304.91688859766435
step 28100:305.01912219578514
step 28200:305.1328907100905
step 28300:305.3700310581847
step 28400:305.2363199348875
step 28500:305.036758206197
step 28600:305.153084389632
step 28700:305.0372236711358
step 28800:304.9077794399902
step 28900:304.7331885651509
step 29000:304.397868040676
step 29100:304.8682811042433
step 29200:305.122285346518
step 29300:305.6900676811342
step 29400:305.81748495786644
step 29500:306.17812300220027
step 29600:306.2718090566766
step 29700:306.1401342712163
step 29800:305.9154266637978
step 29900:306.66757379966367
step 30000:306.8103127919269
step 30100:306.73310350286897
step 30200

step 54800:300.5042507844374
step 54900:300.7033025105667
step 55000:300.75355661701553
step 55100:300.65979203081923
step 55200:300.89434443564625
step 55300:300.87670839617385
step 55400:300.6945425969425
step 55500:300.1993014906551
step 55600:299.83885476556304
step 55700:299.8079588668057
step 55800:299.59118738746525
step 55900:299.51634074244373
step 56000:299.81554328570974
step 56100:300.04675935349667
step 56200:300.10189961139366
step 56300:300.2299185801677
step 56400:300.4849210742587
step 56500:300.48676884121176
step 56600:300.44747730379873
step 56700:300.5277424164454
step 56800:300.30334655882695
step 56900:299.93720744237805
step 57000:299.4795295828559
step 57100:299.25732671789143
step 57200:299.38212490010653
step 57300:299.28395042147713
step 57400:299.2330816622698
step 57500:299.05652458131516
step 57600:298.99134258711973
step 57700:298.81830784686935
step 57800:298.9133874036742
step 57900:298.9655328531227
step 58000:299.12869460510836
step 58100:299.0645710

step 7800:318.9362557877277
step 7900:319.0964649410905
step 8000:318.44126506264047
step 8100:321.60526605561347
step 8200:322.3983231714866
step 8300:321.4736077640052
step 8400:322.4602739348794
step 8500:326.04439629015127
step 8600:324.9560953671295
step 8700:324.7869287086482
step 8800:326.0292053018399
step 8900:326.9570059635771
step 9000:327.13007533105736
step 9100:325.9535171880391
step 9200:328.45339512531285
step 9300:329.76823771615767
step 9400:328.5606203993208
step 9500:330.0876632200004
step 9600:332.261004147648
step 9700:332.5524427883146
step 9800:333.9306471457244
step 9900:333.4616627831091
step 10000:332.08291112135765
step 10100:331.5869197865841
step 10200:330.24073782570935
step 10300:329.34737514267005
step 10400:329.3115438939379
step 10500:328.48743401133186
step 10600:328.3686731394667
step 10700:328.97288710726536
step 10800:327.71391942593243
step 10900:328.8418360473699
step 11000:327.59434193228276
step 11100:327.9697745014916
step 11200:327.743889900

step 36000:298.34424569153083
step 36100:298.3281827249949
step 36200:298.1726772316781
step 36300:298.04636785908764
step 36400:297.63922942961955
step 36500:297.56437365024345
step 36600:297.90686072248656
step 36700:297.81661325432503
step 36800:298.1152785095909
step 36900:298.70977250415876
step 37000:298.56719857685624
step 37100:298.17449646572015
step 37200:297.99411686267746
step 37300:297.87390660051574
step 37400:298.06690785893744
step 37500:298.5649723041514
step 37600:298.90027623169533
step 37700:298.8738765592067
step 37800:299.45022108515263
step 37900:299.5697188986851
step 38000:299.56239154911805
step 38100:299.49531636559817
step 38200:299.4662776476333
step 38300:299.74184193665855
step 38400:299.8706176469676
step 38500:300.1404790607488
step 38600:300.22597014141417
step 38700:300.18828703610876
step 38800:300.1882940877554
step 38900:300.31588488016183
step 39000:300.23601855903524
step 39100:300.19842319254
step 39200:300.44052249511583
step 39300:300.38288775

step 64000:288.7023211325978
step 64100:288.21169121584427
step 64200:287.62265589484656
step 64300:286.77367226568083
step 64400:286.3691496794845
step 64500:286.2281995253597
step 64600:286.24886081762065
step 64700:286.104540683859
step 64800:285.72046704625757
step 64900:285.6175521430179
step 65000:285.43042768538515
step 65100:285.2664496949944
step 65200:284.8066500974386
step 65300:284.6048951428817
step 65400:284.5349826623272
step 65500:284.7158396959609
step 65600:284.8145311119797
step 65700:284.9372237767024
step 65800:285.07686824602786
step 65900:285.18108702111726
step 66000:285.23760922751984
step 66100:285.4917751142053
step 66200:285.60123608238996
step 66300:285.4401603792879
step 66400:285.34328846535436
step 66500:285.2017047843013
step 66600:285.1827475710037
step 66700:285.2954613019696
step 66800:285.2577419543554
step 66900:285.2955016533279
step 67000:285.2176997946945
step 67100:285.36337896881673
step 67200:285.2514643778001
step 67300:285.1117859800501
ste